# 🎮 Pong

Olá e bem vindo ao workshop de **reinforcement learning** (RL) do grupo Turing! O propósito desse projeto é ensinar aos interessados como implementar um algoritmo revolucionário - Q-learning - para resolver um dos maiores clássicos da história dos video-games, pong.

Começaremos falando sobre o problema, ou seja, sobre o jogo Pong. Este que foi o primeiro jogo de video-game lucrativo da história, publicado em 1972, o jogo consta 48 anos de legado. Pong simula uma partida de tênis, existem duas "raquetes" e uma bola, e o objetivo de cada uma das raquetes é não somente evitar que a bola passe por ela, como também fazer com que esta passe pela linha que a outra raquete protege, criando assim a premissa que sustenta o interesse pelo jogo. Queremos então desenvolver um algoritmo capaz de - sem nenhuma explicação adicional - maximizar as suas recompensas, sendo as ações, os estados e as recompensas, todas relativas ao jogo Pong. Teremos no final, portanto, um modelo treinado capaz de bom desempenho dentro do ambiente. 



Usaremos uma biblioteca chamada "numpy" para auxiliar nas computações. Esta é uma biblioteca do python capaz de manuzear diversas computações matemáticas com maestria e nos será importante para o trabalho vindouro.

In [3]:
# Importa a biblioteca Numpy
import numpy as np

Em seguida, é importante definir nossas _variáveis globais_, as quais são responsáveis por regir valores que podem ser ajustados para melhor desempenho do modelo, ou maior velocidade de treinamento. Algumas dessas variáveis apenas farão sentido mais a frente, mas tentaremos dar explicações sucintas para cada uma delas.

* "EPSILON" será o a variável correspondete à letra grega "$\epsilon$" de mesmo nome, que é a notação usual para a probabilidade de exploração do nosso agente. Para exemplificar a posição do nosso agente, que iluminará a função dessa variável, será dado um exemplo do cotidiano humano: Digamos que sua mãe lhe concede algumas moedas de um real para comprar lanches na escola. Sua escola dispõe de quatro máquinas de alimentos, mas todas elas possuem diferentes probabilidades de lhe conceder chocolates de marcas estranhas das quais você nunca ouviu falar. Como você deve aumentar a sua satisfação (conseguir mais frequentemente os chocolates mais gostosos) com um número finito de recursos (as moedas que você possui) sem nunca saber com certeza o que esperar do seu investimento. Esse problema é famoso na área e é conhecido como _the multi-armed bandit problem_, então como resolvê-lo? Usaremos o algoritmo $\epsilon$-guloso, o qual nos instrui a inicialmente explorar bastante até existir certo conhecimento do que esperar de cada máquina de chocolate e, conforme é adquirida experiência, certa confiância de que determinada máquina nos fornece com maior frequencia os chocolates de melhor gosto é criada, então a necessidade de explorar irá decrescer. A variável em questão, portanto, representa a probabilidade inicial de tentarmos uma ação aleatória.

* "EPSILON_MIN", conforme nossa confiança aumenta, será desejável aproveitar nosso conhecimento com maior frequencia do que explorar ainda mais o ambiente. Contudo, sempre fará sentido explorar ao menos um pouco enquanto treinamos o algoritmo e, portanto, é criado um limite inferior para a probabilida de exploração e tal é a função dessa variável.

* "DECAIMENTO", como explicado, é desejado diminuir a exploração conforme a experiência aumenta. Essa variável é justamente a taxa de decaimento da nossa probabilidade de exploração.

* "ALFA", algoritmos de aprendizado de máquina costumam precisar de uma forma de serem otimizados. Q-learning trabalha em cima de gradientes, uma entidade matemática que indica a direção para maximizar (ou minimizar) uma função. Dispondo dessa direção, precisamos informar qual deve ser o tamanho do passo a ser dado antes de atualizar a nova "direção ideal".

* "GAMA" é a variável correspondente a letra grega de mesmo nome "$\gamma$", a qual denota o quanto desejamos que nosso algoritmo considere eventos futuros. Se "$\gamma = 1$", nosso algoritmo avaliará que a situação futura ser melhor que a atual é tão importante quanto a recompensa da situação atual em si, por outro lado, se "$\gamma = 0$", os eventos futuros não apresentam importância alguma para nosso algoritmo. 

* "N_EPISODIOS" dita quantas vezes o agente deverá "reviver" o ambiente (vitórias e derrotas) antes de acabar seu treinamento.

* "Q" é um dicionário, ou seja, uma estrtura de dados capaz de buscar elementos de forma rápida. Nós o usaremos para guardar valores relativos às estimativas do algoritmo.

In [4]:
# Constantes da Política Epsilon Greedy
# Epsilon: probabilidade de experimentar uma ação aleatória
EPSILON = 0.7        # Valor inicial do epsilon
EPSILON_MIN = 0.01   # Valor mínimo de epsilon
DECAIMENTO = 0.98    # Fator de decaímento do epsilon (por episódio)

# Hiperparâmetros do Q-Learning
ALFA = 0.05          # Learning rate
GAMA = 0.9           # Fator de desconto

N_EPISODIOS = 250    # Quantidade de episódios que treinaremos

# Dicionário dos valores de Q
# Chaves: estados; valores: qualidade Q atribuida a cada ação
Q = {}

![](https://www.kdnuggets.com/images/reinforcement-learning-fig1-700.jpg)

Antes de partir para o código do algoritmo em si, alguns esclarecimentos do ponto de vista do agente sobre o ambiente devem ser feitos. Pong é um jogo simples, consiste em controlar a sua raquete com esperança de maximizar a sua pontuação e manter a do adversário a menor possível. Devemos, contudo, adicionar algum rigor à essa expressão, de forma que o algoritmo seja capaz de lidar com essa informação. "controlar a raquete" significa haver três possíveis ações: subir, descer ou não se mover. Cada ação tomada em cada estado deve nos retornar uma recompensa e deve nos levar a um novo estado. A nossa recompensa será de +500 u.r. se pontuarmos, -500 u.r. se pontuarem sobre nós e 0 caso contrário. Finalmente, os estados serão vetores distância entre a raquete a bola, caso jogado no modo fácil.   

In [5]:
# Importando a Biblioteca Gym
import gym

# Criando o nosso Ambiente: Pong
env = gym.make("pong:turing-easy-v0")

# Número total de ações: 3
# 0 = parado; 1 = baixo; 2 = cima
n_acoes = env.action_space.n

print('Número de ações:', n_acoes)

ModuleNotFoundError: No module named 'pygame'

como foi dito, os estados que o nosso agente utilizará para tomar suas decisões são vetores ("setas") que apontam da raquete controlada até a bola, imagine agora o caso onde esses vetores apresentassem apenas módulo como números inteiros. Se a nossa tela possuisse 800 u.d. de largura e 600 u.d. de altura, o espaço amostral dos diferentes vetores e, portanto, diferentes estados do ponto de vista do agente seria $2 \times 800 \times 600 = 960000$. Para um algoritmo que consiste em guardar estimativas do valor de cada ação para cada estado, esse número de estados exigiria não somente guardar como atualizar cada um desses valores até chegar numa estimativa otimizada para os 3 milhões de valores. Não é uma situação ideal. Para simplificar (e agilizar) a situação, "discretizar" os nossos estados é razoável e esperado, faremos com que estados similares o suficiente sejam considerados como iguais e comparilhem das mesmas estimativas (não faz sentido distinguir o vetor (502,234) do vetor (515,222))

In [ ]:
def discretiza_estado(estado):
    return tuple(round(x/10) for x in estado)

TODO: Explicar tomada de ação

In [ ]:
def escolhe_acao(env, Q, estado, epsilon):
    # Se não conhecermos ainda o estado, inicializamos o Q de cada ação como 0
    if estado not in Q.keys(): Q[estado] = [0] * n_acoes

    # Escolhemos um número aleatório com "np.random.random()"
    # Se esse número for menor que epsilon, tomamos uma ação aleatória
    if np.random.random() < epsilon:
        # Escolhemos uma ação aleatória, com env.action_space.sample()
        acao = env.action_space.sample()
    else:
        # Escolhemos a melhor ação para o estado atual, com np.argmax()
        acao = np.argmax(Q[estado])
    return acao

TODO: Explicar função de rodar partida

In [ ]:
def roda_partida(env, Q, renderiza=True):
    # Resetamos o ambiente
    estado = env.reset()

    # Discretizamos o estado
    estado = discretiza_estado(estado)
    
    done = False
    retorno = 0
    
    while not done:
        # Escolhemos uma ação
        acao = escolhe_acao(env, Q, estado, epsilon=0)

        # Tomamos nossa ação escolhida e recebemos informações do próximo estado
        prox_estado, recompensa, done, info = env.step(acao)

        # Discretizamos o próximo estado
        prox_estado = discretiza_estado(prox_estado)

        # Renderizamos o Ambiente
        if renderiza:
            env.render()

        retorno += recompensa
        estado = prox_estado

    print(f'retorno {retorno:.1f},  '
          f'placar {env.score[0]}x{env.score[1]}')
    
    env.close()

In [ ]:
# Rodamos uma partida de Pong
roda_partida(env, Q)

## 🏋️‍♀️ Treinamento

TODO: Falar de Treinamento

TODO: Explicar Bellman (o básico, já foi abordado antes)

In [ ]:
def atualiza_q(Q, estado, acao, recompensa, prox_estado):
    # para cada estado ainda não descoberto, iniciamos seu valor como nulo
    if estado not in Q.keys(): Q[estado] = [0] * n_acoes
    if prox_estado not in Q.keys(): Q[prox_estado] = [0] * n_acoes

    # equação de Bellman
    Q[estado][acao] = Q[estado][acao] + ALFA*(recompensa + GAMA*np.max(Q[prox_estado]) - Q[estado][acao])

TODO: Explicar (brevemente?) pickle

In [ ]:
import pickle

def salva_tabela(Q, nome = 'model.pickle'):
    with open(nome, 'wb') as pickle_out:
        pickle.dump(Q, pickle_out)

def carrega_tabela(nome = 'model.pickle'):
    with open(nome, 'rb') as pickle_out:
        return pickle.load(pickle_out)

TODO: Explicar função de treinamento

In [ ]:
def treina(env, Q):
    retornos = []      # retorno de cada episódio
    epsilon = EPSILON

    for episodio in range(1, N_EPISODIOS+1):
        estado = env.reset()
        estado = discretiza_estado(estado)
        
        done = False
        retorno = 0
        
        while not done:
            # politica
            acao = escolhe_acao(env, Q, estado, epsilon)

            # A ação é tomada e os valores novos são coletados
            # O novo estado é salvo numa nova variavel
            prox_estado, recompensa, done, info = env.step(acao)
            prox_estado = discretiza_estado(prox_estado)

            atualiza_q(Q, estado, acao, recompensa, prox_estado)

            retorno += recompensa
            estado = prox_estado

        epsilon = max(DECAIMENTO*epsilon, EPSILON_MIN)
        retornos.append(retorno)

        if episodio % 10 == 0:
            salva_tabela(Q)

        print(f'episódio {episodio},  '
              f'retorno {retorno:7.1f},  '
              f'retorno médio (últimos 10 episódios) {np.mean(retornos[-10:]):7.1f},  '
              f'placar {env.score[0]}x{env.score[1]},  '
              f'epsilon: {epsilon:.3f}')
        
    env.close()

In [ ]:
treina(env, Q)

## 🏓 Testando nosso Agente Treinado

In [ ]:
roda_partida(env, Q)